In [1]:
# if run on colab
#!pip install torcheeg
#from google.colab import mount
#drive.mount('/content/drive', force_remount=True) 

In [2]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

In [3]:
dataset = BCICIV2aDataset(
    root_path='./BCICIV-2a-mat/',
    io_path=f'./examples_pipeline/bciciv-2a',
    # skip_trial_with_artifacts=True,
    # offline_transform=transforms.Compose([
    #     transforms.BandDifferentialEntropy(apply_to_baseline=True),
    #     transforms.To2d(apply_to_baseline=True),
    #     transforms.ToTensor(apply_to_baseline=True)
    # ]),
    online_transform=transforms.Compose([
        # transforms.To2d(apply_to_baseline=True),
        # transforms.ToTensor(apply_to_baseline=True),
        transforms.To2d(),
        transforms.ToTensor(),
        # transforms.CWTSpectrum(apply_to_baseline=True),
        # transforms.BandDifferentialEntropy(apply_to_baseline=True),
        # transforms.BaselineRemoval(),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-13 16:28:06] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./examples_pipeline/bciciv-2a.
[2024-05-13 16:28:06] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|                                                                                                                       | 0/18 [00:00<?, ?it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 0it [00:00, ?it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 1it [00:00,  2.10it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 41it [00:00, 93.57it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 76it [00:00, 156.92it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 113it [00:00, 212.32it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 145it [00:00, 238.91it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 182it [00:00, 273.45it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 216it [00:01, 290.80it/s]
[RECORD ./BCICIV-2a-ma

In [4]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A06E_0        A06         0       E     A06    3   
1         2254    4004    A06E_1        A06         1       E     A06    3   
2         4172    5922    A06E_2        A06         2       E     A06    3   
3         6124    7874    A06E_3        A06         3       E     A06    3   
4         8132    9882    A06E_4        A06         4       E     A06    3   
...        ...     ...       ...        ...       ...     ...     ...  ...   
5179     86751   88501  A05T_283        A05        43       T     A05    8   
5180     88657   90407  A05T_284        A05        44       T     A05    8   
5181     90585   92335  A05T_285        A05        45       T     A05    8   
5182     92699   94449  A05T_286        A05        46       T     A05    8   
5183     94758   96508  A05T_287        A05        47       T     A05    8   

      label  _record_id  
0         1   _recor

In [5]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [ ]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

model = ATCNet(
    num_classes=4,
    num_electrodes=22,
    chunk_size=7*250,
)

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4
    )
    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader, 
        max_epochs=10,
        default_root_dir=f'./examples_pipeline/atcnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-13 16:29:39] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-05-13 16:29:39] INFO (torcheeg/MainThread) 😊 | Please set split_path to ./examples_pipeline/split for the next run, if you want to use the same setting for the experiment.
/home/quang/yes/envs/cognitive-computing/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1712608958871/work/aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv2d(input, weight, bias, self.stride,
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/quang/yes/envs/cognitive-computing/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboard

Epoch 0:  74%|██████████████████████████████████████████████▎                | 50/68 [02:54<01:02,  0.29it/s, v_num=0, train_loss=1.390, train_accuracy=0.250]

In [ ]:
atc_weights_path = './weights/atc_weights.pt'
torch.save(model.state_dict(), atc_weights_path)
# to load weight:
# model.load_state_dict(torch.load(atc_weights_path))